# Classifying handwritten digits with Convolutional Neural Networks
### using Tensorflow
useful links:
- https://www.tensorflow.org/versions/r0.11/tutorials/deep_cnn/index.html


Below is some generic Code for setting up an interactive tensorflow session and the mnist dataset.

In [9]:
import tensorflow as tf
from functools import *
import time
from tensorflow.examples.tutorials.mnist import input_data
#onehot means a vector is 1 in a single dimension and 0 in every other
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Model

In [46]:
n_inputs = 784
n_classes = 10
batch_size = 50

x = tf.placeholder('float', [None, n_inputs])
y = tf.placeholder('float',[None, 10])


weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 16])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 16, 32])),
    'wd1': tf.Variable(tf.random_normal([7*7*32, 512])),
    'out': tf.Variable(tf.random_normal([512, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([16])),
    'bc2': tf.Variable(tf.random_normal([32])),

    'bd1': tf.Variable(tf.random_normal([512])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [4]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

In [5]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [49]:
def conv_neural_net(x, weights, biases):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    
    # Pooling 1
    conv1 = maxpool2d(conv1, k=2)

    # Convolution 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    
    # Pooling 2
    conv2 = maxpool2d(conv2, k=2)
    
    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)

    # Predictions
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])

    return out

In [44]:
def train_and_test_conv_neural_net(x, y, weights, biases, epochs = 1):
    
    pred = conv_neural_net(x, weights, biases)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
    opt = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for e in range(epochs):
            epoch_loss = 0
            for i in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([opt, cost], 
                                feed_dict = {x:epoch_x, y:epoch_y})
                epoch_loss = epoch_loss + c
            print('Epoch ' + str(e+1) + ' out of ' + 
            str(epochs) + ' / Loss: ' + str(epoch_loss))

        print 'Training finished!'

        correct = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        print 'Accuracy: ' + str(accuracy.eval({x:mnist.test.images, 
                                                y:mnist.test.labels}))

## Training

In [ ]:
start_time = time.time()
train_and_test_conv_neural_net(x, y, weights, biases, 1)
print 'Time: ' +  str(time.time() - start_time)

Epoch 1 out of 1 / Loss: 1183012.49048
Training finished!
